In [40]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [41]:
#Load the Trained Model scaler, pickle, onehotencoder

model = load_model('model.h5')

#Load encoder and scaler

with open('onehot_encoder_geo.pkl','rb') as file:
    one_encoder_geo = pickle.load(file)

with open('label_encoder.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)    

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [42]:
#Example input data

input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [43]:
#Onehot encode geography

geo_encoded = one_encoder_geo.transform([[input_data.get('Geography')]]).toarray()
geo_encoded
geo_encoded_df = pd.DataFrame(geo_encoded, columns= one_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/sateesh.chikkalagi/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [44]:
#Convert input data into dataframe
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [45]:
#Encode Categorical Values
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [46]:
#concationation one hot encoded and drop Gepgraphy

input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [47]:
#Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [48]:
#Prediction Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[0.049224]], dtype=float32)

In [49]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.049223997)

In [50]:
if prediction_proba > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")    

The customer is not likely to churn
